# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [45]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [46]:
# Use this cell for any import statements you add
import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [47]:
artists = {'jason_isbell':"https://www.azlyrics.com/j/jasonisbell.html",
           'sticky_fingers':"https://www.azlyrics.com/s/stickyfingers.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: Yes, the scraping is allowed by this page. The robots.txt page on www.azlyrics.com gives useragent is 008, meaning that all web scraping activity is allowed. 

In [48]:
# Intialize Dictionary 
lyrics_pages = defaultdict(list)

# Initilize regex pattern
pattern = r'^/lyrics/.*$' 
# looks for items that include /lyrics/ at the beginning of the string (^), 
# with any number of characters (.*) until the end of the string ($)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())
    print(r)
    soup = BeautifulSoup(r.content, "html.parser")
    # Loop through the links on the page and add them to the filtered list if they match the regex
    for link in soup.find_all('a'):
        if link is not None and re.match(pattern, str(link.get('href'))):
            lyrics_pages[artist].append(link.get('href'))

<Response [200]>
<Response [200]>


Let's make sure we have enough lyrics pages to scrape. 

In [49]:
for artist, lp in lyrics_pages.items():
    assert(len(set(lp)) > 20) 
    # the assert statement will raise an AssertionError if the condition is not true 

In [50]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items(): 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For jason_isbell we have 110.
The full pull will take for this artist will take 0.31 hours.
For sticky_fingers we have 84.
The full pull will take for this artist will take 0.23 hours.


In [51]:
# Add prefix to each to create a full URL 
url_stub = "https://www.azlyrics.com" 

# Add the url stub to the beginning of each URL page in the dictionary
for artist, pages in lyrics_pages.items():
    lyrics_pages[artist] = [url_stub + url for url in pages]

## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [52]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [53]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")
os.mkdir("lyrics")

In [54]:
# Reduce dictionary to only first 20 songs for each artist - full amount would take too long 
for artist, pages in lyrics_pages.items():
    lyrics_pages[artist] = pages[:20]

In [55]:
start = time.time()

total_pages = 0 

for artist, pages in lyrics_pages.items():
    
    # Use this space to carry out the following steps: 
    
    # 1. Build a subfolder for the artist
    artist_folder = os.path.join("lyrics", artist)
    if os.path.isdir(artist_folder):  # If the folder already exists, delete it 
        shutil.rmtree(artist_folder)
    os.mkdir(artist_folder)

    # 2. Iterate over the lyrics pages
    for page in pages:
        # 3. Request the lyrics page. 
        r = requests.get(page)
        time.sleep(5 + 10*random.random())
        print(r)
        
        # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(r.content, "html.parser")
        title = soup.find_all("b")[1].text.strip()
        lyrics = ""
        for div in soup.find_all("div", attrs={"class": None, "id": None}):
            if div.text.strip():
                lyrics += div.text.strip() + "\n"

        # 5. Write out the title, two returns ('\n'), and the lyrics.
        filename = generate_filename_from_link(page)
        with open(os.path.join(artist_folder, filename), "w") as f:
            f.write(title + "\n\n" + lyrics + "\n\n")
            
    total_pages += len(pages)
    print(f"{total_pages} pages saved for {artist}")
    
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
20 pages saved for jason_isbell
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
40 pages saved for sticky_fingers
Total run time was 0.13 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [56]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [57]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For jason_isbell we have 20 files.
For jason_isbell we have roughly 5093 words, 953 are unique.
For sticky_fingers we have 19 files.
For sticky_fingers we have roughly 3684 words, 871 are unique.
